<a href="https://colab.research.google.com/github/Congliu0829/cencus_dnn_caching/blob/master/Census_income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline
import os
import time
from sklearn import preprocessing
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
# cpu

Populating the interactive namespace from numpy and matplotlib


In [0]:
# import math
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# %pylab inline
# import os
# import time
# from sklearn import preprocessing
# from numpy import array
# from numpy import argmax
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder
# from sklearn import preprocessing
# from os.path import exists
# from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
# platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
# cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
# accelerator = cuda_output[1] if exists('/dev/nvidia0') else 'cpu'
# !pip install -f https://download.pytorch.org/whl/torch_stable.html torchvision
# cuda

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'


import torch
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
# !pip uninstall torch==0.4.1
!pip install torch==0.4.1
!pip install torchvision==0.2.2
!pip install Pillow==6.2.2
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import pickle
from torch.utils.data.sampler import (SubsetRandomSampler,RandomSampler)
# cpu

In [0]:

# # http://pytorch.org/
# from os.path import exists
# from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
# platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
# cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
# accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

# !pip install -f https://download.pytorch.org/whl/torch_stable.html torchvision
# import torch
# # http://pytorch.org/
# from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
# platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
# cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
# accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader
# import pickle
# from torch.utils.data.sampler import (SubsetRandomSampler,RandomSampler)
# from torchvision import datasets, transforms
# cuda

In [0]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    use_cuda = True
else:
    device = torch.device("cpu")
    use_cuda = False
    
print(device)

cpu


In [0]:
full_data = pd.read_csv(
    "/content/adult.csv",
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python', skiprows=1,
        na_values="?", dtype={0:int, 1:str, 2:int, 3:str, 4:int, 5: str, 6:str , 7:str ,8:str ,9: str, 10:int, 11:int, 12:int, 13:str,14: str})

print('Dataset size: ', full_data.shape[0])

Dataset size:  32561


In [0]:
print(full_data.head())
print(full_data.shape)

   Age  Workclass  fnlwgt  ... Hours per week          Country   Target
0   90        "?"   77053  ...             40  "United-States"  "<=50K"
1   82  "Private"  132870  ...             18  "United-States"  "<=50K"
2   66        "?"  186061  ...             40  "United-States"  "<=50K"
3   54  "Private"  140359  ...             40  "United-States"  "<=50K"
4   41  "Private"  264663  ...             40  "United-States"  "<=50K"

[5 rows x 15 columns]
(32561, 15)


In [0]:
str_list=[] 
for data in [full_data]:
    for colname, colvalue in data.iteritems(): 
        if type(colvalue[1]) == str:
            str_list.append(colname) 
num_list = data.columns.difference(str_list) #seperate str with int in input data

In [0]:
full_size = full_data.shape[0]
print('Dataset size Before pruning: ', full_size)
for data in [full_data]:
    for i in full_data:
        data[i].replace('NaN', np.nan, inplace=True)
    data.dropna(inplace=True)
real_size = full_data.shape[0]
print('Dataset size after pruning: ', real_size)
print('We eliminated ', (full_size-real_size), ' datapoints')

Dataset size Before pruning:  32561
Dataset size after pruning:  32561
We eliminated  0  datapoints


In [0]:
full_data.head()

,Age,Workclass,fnlwgt,Education,Education-Num,Martial Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,90,"""?""",77053,"""HS-grad""",9,"""Widowed""","""?""","""Not-in-family""","""White""","""Female""",0,4356,40,"""United-States""","""<=50K"""
1,82,"""Private""",132870,"""HS-grad""",9,"""Widowed""","""Exec-managerial""","""Not-in-family""","""White""","""Female""",0,4356,18,"""United-States""","""<=50K"""
2,66,"""?""",186061,"""Some-college""",10,"""Widowed""","""?""","""Unmarried""","""Black""","""Female""",0,4356,40,"""United-States""","""<=50K"""
3,54,"""Private""",140359,"""7th-8th""",4,"""Divorced""","""Machine-op-inspct""","""Unmarried""","""White""","""Female""",0,3900,40,"""United-States""","""<=50K"""
4,41,"""Private""",264663,"""Some-college""",10,"""Separated""","""Prof-specialty""","""Own-child""","""White""","""Female""",0,3900,40,"""United-States""","""<=50K"""


In [0]:
full_labels = full_data['Target'].copy()
print(full_labels.shape[0])

full_data = full_data.drop(['Target'], axis=1)
print(full_data.shape[0])

# Label Encode Labels
label_encoder = LabelEncoder()
full_labels = label_encoder.fit_transform(full_labels)
print(full_labels[:10])

32561
32561
[0 0 0 0 0 0 0 1 0 1]


In [0]:
cat_data = full_data.select_dtypes(include=['object']).copy()
other_data = full_data.select_dtypes(include=['int']).copy()
print(cat_data.head())
print(cat_data.shape)
print(other_data.shape)#seperate data by objects and numbers 

   Workclass       Education Martial Status  ...     Race       Sex          Country
0        "?"       "HS-grad"      "Widowed"  ...  "White"  "Female"  "United-States"
1  "Private"       "HS-grad"      "Widowed"  ...  "White"  "Female"  "United-States"
2        "?"  "Some-college"      "Widowed"  ...  "Black"  "Female"  "United-States"
3  "Private"       "7th-8th"     "Divorced"  ...  "White"  "Female"  "United-States"
4  "Private"  "Some-college"    "Separated"  ...  "White"  "Female"  "United-States"

[5 rows x 8 columns]
(32561, 8)
(32561, 6)


In [0]:
newcat_data = pd.get_dummies(cat_data, columns=[
    "Workclass", "Education", "Country" ,"Relationship", "Martial Status", "Occupation", "Relationship",
    "Race", "Sex"
])# apply a encoder for str catagories

In [0]:
newcat_data.head()

,"Workclass_""?""","Workclass_""Federal-gov""","Workclass_""Local-gov""","Workclass_""Never-worked""","Workclass_""Private""","Workclass_""Self-emp-inc""","Workclass_""Self-emp-not-inc""","Workclass_""State-gov""","Workclass_""Without-pay""","Education_""10th""","Education_""11th""","Education_""12th""","Education_""1st-4th""","Education_""5th-6th""","Education_""7th-8th""","Education_""9th""","Education_""Assoc-acdm""","Education_""Assoc-voc""","Education_""Bachelors""","Education_""Doctorate""","Education_""HS-grad""","Education_""Masters""","Education_""Preschool""","Education_""Prof-school""","Education_""Some-college""","Country_""?""","Country_""Cambodia""","Country_""Canada""","Country_""China""","Country_""Columbia""","Country_""Cuba""","Country_""Dominican-Republic""","Country_""Ecuador""","Country_""El-Salvador""","Country_""England""","Country_""France""","Country_""Germany""","Country_""Greece""","Country_""Guatemala""","Country_""Haiti""",...,"Relationship_""Not-in-family""","Relationship_""Other-relative""","Relationship_""Own-child""","Relationship_""Unmarried""","Relationship_""Wife""","Martial Status_""Divorced""","Martial Status_""Married-AF-spouse""","Martial Status_""Married-civ-spouse""","Martial Status_""Married-spouse-absent""","Martial Status_""Never-married""","Martial Status_""Separated""","Martial Status_""Widowed""","Occupation_""?""","Occupation_""Adm-clerical""","Occupation_""Armed-Forces""","Occupation_""Craft-repair""","Occupation_""Exec-managerial""","Occupation_""Farming-fishing""","Occupation_""Handlers-cleaners""","Occupation_""Machine-op-inspct""","Occupation_""Other-service""","Occupation_""Priv-house-serv""","Occupation_""Prof-specialty""","Occupation_""Protective-serv""","Occupation_""Sales""","Occupation_""Tech-support""","Occupation_""Transport-moving""","Relationship_""Husband""","Relationship_""Not-in-family""","Relationship_""Other-relative""","Relationship_""Own-child""","Relationship_""Unmarried""","Relationship_""Wife""","Race_""Amer-Indian-Eskimo""","Race_""Asian-Pac-Islander""","Race_""Black""","Race_""Other""","Race_""White""","Sex_""Female""","Sex_""Male"""
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0


In [0]:
full_data = pd.concat([other_data, newcat_data], axis=1)

In [0]:
print(full_data.head())

   Age  fnlwgt  Education-Num  ...  Race_"White"  Sex_"Female"  Sex_"Male"
0   90   77053              9  ...             1             1           0
1   82  132870              9  ...             1             1           0
2   66  186061             10  ...             0             1           0
3   54  140359              4  ...             1             1           0
4   41  264663             10  ...             1             1           0

[5 rows x 114 columns]


In [0]:
train_size = 20000
valid_size = 10000

In [0]:
train_x = full_data.iloc[:train_size, :]
train_y = full_labels[:train_size]
print(train_x.shape)
print(train_y.shape)
print()

valid_x = full_data.iloc[train_size:(train_size+valid_size), :]
valid_y = full_labels[train_size:(train_size+valid_size)]
print(valid_x.shape)
print(valid_y.shape)
print()

test_x = full_data.iloc[(train_size+valid_size):, :]
test_y = full_labels[(train_size+valid_size):]
print(test_x.shape)
print(test_y.shape)
num_features = test_x.shape[1]
print(type(train_x))

(20000, 114)
(20000,)

(10000, 114)
(10000,)

(2561, 114)
(2561,)
<class 'pandas.core.frame.DataFrame'>


In [0]:
print(train_y[:25])

[0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]


In [0]:
num_features

114

In [0]:
directory = os.path.dirname('./data_ML/')

try:
    os.stat(directory)
except:
    os.mkdir(directory)

In [0]:
import os, shutil
folder = './data_ML/'
for the_file in os.listdir(folder):
    file_path = os.path.join(folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

!ls "./data_ML/"

In [0]:
full_dataset = np.asarray(full_data).astype(np.float32)
z=0

# load data for dataloader
start = time.time()
for x in range(full_dataset.shape[0]):
    for y in range(2):
        if full_labels[x] == y: 
            temp = (full_dataset[x,:])
            
            directory = './data_ML/' + str(label_encoder.classes_[y])
            if not os.path.exists(directory):
                os.makedirs(directory)
            
            
            np.save((directory+ '/' +str(z) +'.npy'), temp)
            
            z += 1
    
end = time.time()

print('Time to process: ', end-start)
print(z, ' datapoints saved to path')

Time to process:  7.872073411941528
32561  datapoints saved to path


In [0]:
mean = np.zeros(num_features)
std_mean = np.ones(num_features)


class prep_stuff:
    def __init__(self, batch_size, path):
        self.batch_size = batch_size
        self.path = path

        batch_size_eval = 128
          
        self.train_data = datasets.DatasetFolder(self.path, np.load, ('npy') 
                               )
   
        
        indices = list(range(len(self.train_data)))
        random.shuffle(indices)

        # Split dataset into train and Test sets
        self.train_loader = DataLoader(
            self.train_data,
            batch_size=self.batch_size,
            sampler=SubsetRandomSampler(indices[:20000]),
            num_workers=1,
        )

        self.valid_loader = DataLoader(
            self.train_data,
            batch_size=self.batch_size,
            sampler=SubsetRandomSampler(indices[20000:30000]),
            num_workers=1,
        )

        self.test_loader = DataLoader(
            self.train_data,
            batch_size=batch_size_eval,
            sampler=SubsetRandomSampler(indices[30000:]),
            num_workers=1,
        )
        
           


# Create the object with both loader and loss functions

batchSize = 1000
# batchSize = 1
path = './data_ML/'

# 20000 for train
# 10000 for valid 
# 2561 for Test
pre_model = prep_stuff(batchSize, path)

In [0]:
# two intermediate layers network
class BasicNet(nn.Module):
    
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_features = num_features
        self.num_classes = num_classes
        self.layers = 0
        
        self.lin1 = torch.nn.Linear(self.num_features,  150)        
        self.lin2 = torch.nn.Linear(150, 150)        
        self.lin3 = torch.nn.Linear(150, 150) 
        self.lin4 = torch.nn.Linear(150, self.num_classes)
        
        self.prelu = nn.PReLU()
        self.dropout = nn.Dropout(0.25)
        self.cachelin1 = []
        self.cachelin2 = []
        self.cache_label = []



    # def EncoderDecoder(self, xin):
    #     dim = xin.size(1)
    #     x = torch.nn.Linear(dim, 10)(xin)
    #     self.cache.append(x)
    #     x = torch.nn.Linear(10, dim)(x)
    #     return x


    def forward(self, xin):
        self.layers = 0
        x = F.relu(self.lin1(xin)) #input layer
        #should be one cache
        self.layers += 1
        
        # for y in range(8):
        #   x = F.relu(self.lin4(x)) 
        #   self.cache.append(x)
        #   self.layers += 1

        x = F.relu(self.lin2(x))     #intermediate layer1
        self.cachelin1.append(x)
        
        x = F.relu(self.lin3(x))     #intermediate layer2
        self.cachelin2.append(x)

        # x = self.dropout(x)
        # self.cache.append(x)
        # x = self.EncoderDecoder(x)
        
        x = F.relu(self.lin4(x)) #output layer
        return x




In [0]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target)
        # Backprop
        loss.backward(retain_graph=True)
        optimizer.step()

In [0]:
def test(model, test_loader):
    model.eval()
    
    test_loss = 0
    correct = 0
    test_size = 0
    
    with torch.no_grad():
      
        for inputs, target in test_loader:
            inputs, target = inputs.to(device), target.to(device)
            output = model(inputs)
            test_size += len(inputs)
            test_loss += test_loss_fn(output, target).item() 
            pred = output.max(1, keepdim=True)[1] 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= test_size
    accuracy = correct / test_size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, test_size,
        100. * accuracy))
    
    return test_loss, accuracy

In [0]:
model = BasicNet(num_features, 2).to(device)
test_accuracy = []
train_loss = []
nbr_epochs = 10
lr = 0.0025
weight_decay = 0


# Surrogate loss used for training
loss_fn = nn.CrossEntropyLoss()
test_loss_fn = nn.CrossEntropyLoss(reduction='sum')

optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
# optimizer = optim.SGD(model.parameters(), lr=lr ,weight_decay=weight_decay)
#optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)


print('Training beginning...')
start_time = time.time()
cache_input = []
# cache_output = []
for epoch in range(1, nbr_epochs+1):
    print('Epoch ', epoch, ':')
    for inputs, target in pre_model.train_loader:
      inputs, target = inputs.to(device), target.to(device)
      cache_input.append(inputs)

      optimizer.zero_grad()
      output = model(inputs)  
      # cache_output.append(target)
      model.cache_label.append(target)
      loss = loss_fn(output, target)
      loss.backward(retain_graph=True)
      optimizer.step()
    # train(model, pre_model.train_loader, optimizer, epoch)

    # loss, acc = test(model, pre_model.valid_loader)
    
    # save results every epoch
    
    # test_accuracy.append(acc)
    # train_loss.append(loss)
    
end_time = time.time()
print('Training on ' + str(nbr_epochs) + ' epochs done in ', str(end_time-start_time),' seconds')

Training beginning...
Epoch  1 :
Epoch  2 :
Epoch  3 :
Epoch  4 :
Epoch  5 :
Epoch  6 :
Epoch  7 :
Epoch  8 :
Epoch  9 :
Epoch  10 :
Training on 10 epochs done in  52.629416704177856  seconds


In [0]:
# print(model.cache_label[0].size())
print(type(model.cache_label))
print(len(model.cache_label))
print(model.cache_label[0].size())

<class 'list'>
200
torch.Size([1000])


In [0]:
#take out the last set of cache
model.cache_label = model.cache_label[(len(model.cache_label)-int(train_size/batchSize)):len(model.cache_label)]
print(len(model.cache_label))
cachelabel = []
for i in model.cache_label:
 cachelabel.append(i.detach().numpy())
for i in cachelabel:
  i.flatten()
cache_whole_label = []
for i in cachelabel:
  for m in i:
    cache_whole_label.append(m)
########################################################cache_whole_label is whole labels of traindata

20


In [0]:
len(cache_whole_label)

20000

In [0]:
# # 对每一个cache（一共2个）都建立一个网络来做降纬处理
# class EncoderDecoder1(nn.Module):
#     def __init__(self, in_dim, out_dim): 
#         super(EncoderDecoder,self).__init__()
#         self.linear1 = torch.nn.Linear(in_dim,50) 
#         self.linear2 = torch.nn.Linear(50,50)
#         self.linear3 = torch.nn.Linear(50,20)
#         self.linear4 = torch.nn.Linear(20,10)
#         self.cache_mdim = [] #New trained cache
#         self.linear5 = torch.nn.Linear(10,out_dim)

#     def forward(self,x):
#         x = self.linear1(x)
#         x = F.relu(x)
#         x = self.linear2(x)
#         x = F.relu(x)
#         x = self.linear3(x)
#         x = F.relu(x)
#         x = self.linear4(x)
#         x = F.relu(x)
#         self.cache_mdim.append(x) 
#         x = self.linear5(x)

#         return x

# # for i in CACHE_LIST:
# #   i = i.view(-1,150)

# coder = EncoderDecoder(150, 150).to(device)



In [0]:
# construct encoder&decoder for each cache (2 in total)
class EncoderDecoder1(nn.Module):
    def __init__(self, in_dim, out_dim): 
        super(EncoderDecoder1,self).__init__()
        self.linear1 = torch.nn.Linear(in_dim,50) 
        self.linear2 = torch.nn.Linear(50,4)
        # self.linear3 = torch.nn.Linear(10,4)
        # self.linear4 = torch.nn.Linear(20,10)
        self.cache_mdim1 = [] #New trained cache
        self.linear3 = torch.nn.Linear(4,out_dim)

    def forward(self,x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        self.cache_mdim1.append(x)
        x = self.linear3(x)
        x = F.relu(x)
 
        # x = self.linear4(x)
        # x = F.relu(x)


        return x

# for i in CACHE_LIST:
#   i = i.view(-1,150)

coder1 = EncoderDecoder1(150, 150).to(device)


class EncoderDecoder2(nn.Module):
    def __init__(self, in_dim, out_dim): 
        super(EncoderDecoder2,self).__init__()
        self.linear1 = torch.nn.Linear(in_dim,50) 
        self.linear2 = torch.nn.Linear(50,4)
        # self.linear3 = torch.nn.Linear(10,4)
        # self.linear4 = torch.nn.Linear(20,10)
        self.cache_mdim2 = [] #New trained cache
        self.linear3 = torch.nn.Linear(4,out_dim)

    def forward(self,x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        self.cache_mdim2.append(x) 
        x = self.linear3(x)
        x = F.relu(x)

        # x = self.linear4(x)
        # x = F.relu(x)


        return x

# for i in CACHE_LIST:
#   i = i.view(-1,150)

coder2 = EncoderDecoder2(150, 150).to(device)



In [0]:

#take out the last set of original cache from basic network
model.cachelin1 = model.cachelin1[(len(model.cachelin1)- int(train_size/batchSize)):len(model.cachelin1)]
model.cachelin2 = model.cachelin2[(len(model.cachelin2)- int(train_size/batchSize)):len(model.cachelin2)]
print(len(model.cachelin1))


20


In [0]:
#convert tensors in cache from basic newtork to individual tensor

#convert tensors to numpy
cachelin1_np = []
for i in model.cachelin1:
 cachelin1_np.append(i.detach().numpy())
for i in cachelin1_np:
  i.flatten()
cachelin1 = []
for i in cachelin1_np:
  for m in i:
    cachelin1.append(m)


cachelin2_np = []
for i in model.cachelin2:
 cachelin2_np.append(i.detach().numpy())
for i in cachelin2_np:
  i.flatten()
cachelin2 = []
for i in cachelin2_np:
  for m in i:
    cachelin2.append(m)

#convert numpy to tensors
for i in range (len(cachelin1)):
  cachelin1[i] = torch.from_numpy(cachelin1[i])

for i in range (len(cachelin2)):
  cachelin2[i] = torch.from_numpy(cachelin2[i])

In [0]:
cachelin1[0].size()

torch.Size([150])

In [0]:
# epoch_dim = 10
# criterion = nn.MSELoss()
# optimizer = optim.Adam(coder.parameters(),lr=0.01)
# for epoch in range(epoch_dim):
#   running_loss = 0.0
#   for i in CACHE_LIST:
#     for m in i:
#       inputs = m.to(device)
#       optimizer.zero_grad()
#       outputs = coder(inputs)
#       loss = criterion(outputs, inputs)
#       loss.backward(retain_graph=True)
#       running_loss += loss.item()


    
# print('Finish Training')


In [0]:
# training to reduce dimsension of first cache
epoch_dim = 10
criterion = nn.MSELoss()
optimizer = optim.Adam(coder1.parameters(),lr=0.01)
for epoch in range(epoch_dim):
  running_loss = 0.0
  for i in cachelin1:
    inputs = i.to(device)
    optimizer.zero_grad()
    outputs = coder1(inputs)
    loss = criterion(outputs, inputs)
    loss.backward(retain_graph=True)
    running_loss += loss.item()
      


    
print('Finish Training1')


Finish Training1


In [0]:
# training to reduce dimension of second cache
epoch_dim = 10
criterion = nn.MSELoss()
optimizer = optim.Adam(coder2.parameters(),lr=0.01)
for epoch in range(epoch_dim):
  running_loss = 0.0
  for i in cachelin2:
    inputs = i.to(device)
    optimizer.zero_grad()
    outputs = coder2(inputs)
    loss = criterion(outputs, inputs)
    loss.backward(retain_graph=True)
    running_loss += loss.item()


    
print('Finish Training2')


Finish Training2


In [0]:
print(len(coder1.cache_mdim1))
print(len(coder2.cache_mdim2))

200000
200000


In [0]:
#take the last set of reduced-dimension cache respectively
cache_mdim1 = []
cache_mdim2 = []
cache_mdim1 = coder1.cache_mdim1[(len(coder1.cache_mdim1)-int(train_size)):len(coder1.cache_mdim1)]
cache_mdim2 = coder2.cache_mdim2[(len(coder2.cache_mdim2)-int(train_size)):len(coder2.cache_mdim2)]


In [0]:
#convert reduced-dimension cache to numpy for k-means respectively
cache_mdim1_np = []
cache_mdim2_np = []
for i in range(len(cache_mdim1)):
  cache_mdim1_np.append(cache_mdim1[i].detach().numpy())

for i in range(len(cache_mdim2)):
  cache_mdim2_np.append(cache_mdim2[i].detach().numpy())
######################################

In [0]:
#cache_whole_label代表总最后一组的label，cachelin1,cachelin2代表最后一组的储存的tensor，cache_mdim1和cache_midm2分别为前面的cache经过降维处理的tensor
#cache_whole_label(list,20000,每个都是一个数字，type:<class 'numpy.int64'>), 
#cachelin1,cachelin2(list,20000,每一个都是一个tensor，torch.Size([150]))
#cache_mdim1,cache_mdim2 (list,20000,每一个都是一个tensor,torch.Size([4]))

In [0]:
#kmeans
# data1 = cache_mdim1_np
# data2 = cache_mdim2_np
# data1 = np.array(data1)
# data2 = np.array(data2)
# #normalization
# def normalize(data):
#     m = np.mean(data)
#     mx = max(data)
#     mn = min(data)
#     return [(float(i) - m) / (mx - mn) for i in data]

# for i in range(len(data1)):
#   data1[i] = normalize(data1[i])
# for i in range(len(data2)):
#   data2[i] = normalize(data2[i])
# from sklearn.cluster import KMeans
# kmeans1 = KMeans(2, random_state=0).fit(data1)
# kmeans2 = KMeans(2, random_state=0).fit(data2)


# print(kmeans1.cluster_centers_)
# print(kmeans2.cluster_centers_)
# num = 0
# for i in kmeans1.labels_:
#   if i != 0:
#     num = num+1
# print(num)

In [0]:
##example of kmeans from sklearn

# >>> from sklearn.cluster import KMeans
# >>> import numpy as np
# >>> X = np.array([[1, 2], [1, 4], [1, 0],
# ...               [4, 2], [4, 4], [4, 0]])
# >>> kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
# >>> kmeans.labels_
# array([0, 0, 0, 1, 1, 1], dtype=int32)
# >>> kmeans.predict([[0, 0], [4, 4]])
# array([0, 1], dtype=int32)
# >>> kmeans.cluster_centers_
# array([[ 1.,  2.],
#        [ 4.,  2.]])


In [0]:
#original cache from basic data
cachelin1
cachelin2

In [0]:
len(cachelin1)

20000

In [0]:
#reduced dimension cache from encoder&decoder
cache_mdim1
cache_mdim2

In [0]:
cache_mdim1[0].size()

torch.Size([4])

In [0]:
len(cache_mdim1)

20000

In [0]:
#kmeans centroid
# kmeans1.cluster_centers_
# kmeans2.cluster_centers_

In [0]:

# # from sklearn.neighbors import KNeighborsClassifier
# from sklearn.neighbors import NearestNeighbors
# #核心代码
# knn_layer1=KNeighborsClassifier(n_neighbors=10)
# knn_layer1.fit(cache_mdim1_np,cache_whole_label)                      #用训练集进行训练模型
 
# knn_layer2=KNeighborsClassifier(n_neighbors=10)
# knn_layer2.fit(cache_mdim2_np,cache_whole_label)   

 
#核心代码
# prediction=knn.predict(x)
 
# print(prediction)


In [0]:
# two intermediate layers network
class TestNet(nn.Module):
    
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_features = num_features
        self.num_classes = num_classes
        self.layers = 0
        
        self.lin1 = torch.nn.Linear(self.num_features,  150)        
        self.lin2 = torch.nn.Linear(150, 150)        
        self.lin3 = torch.nn.Linear(150, 150) 
        self.lin4 = torch.nn.Linear(150, self.num_classes)
        
        self.prelu = nn.PReLU()
        # self.dropout = nn.Dropout(0.25)
        self.testcachelin1 = []
        self.testcachelin2 = []
        self.testcache_label = []



    # def EncoderDecoder(self, xin):
    #     dim = xin.size(1)
    #     x = torch.nn.Linear(dim, 10)(xin)
    #     self.cache.append(x)
    #     x = torch.nn.Linear(10, dim)(x)
    #     return x


    def forward(self, xin):
        # self.layers = 0
        x = F.relu(self.lin1(xin)) #input layer
        #should be one cache
        # self.layers += 1
        
        # for y in range(8):
        #   x = F.relu(self.lin4(x)) 
        #   self.cache.append(x)
        #   self.layers += 1

        x = F.relu(self.lin2(x))     #intermediate layer1
        self.testcachelin1.append(x)
        
        x = F.relu(self.lin3(x))     #intermediate layer2
        self.testcachelin2.append(x)

        # x = self.dropout(x)
        # self.cache.append(x)
        # x = self.EncoderDecoder(x)
        
        x = F.relu(self.lin4(x)) #output layer
        # self.layers += 1
        return x




In [0]:
testmodel = TestNet(num_features, 2).to(device)
test_accuracy = []
train_loss = []
nbr_epochs = 10
lr = 0.0025
weight_decay = 0


# Surrogate loss used for training
loss_fn = nn.CrossEntropyLoss()
test_loss_fn = nn.CrossEntropyLoss(reduction='sum')

optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
# optimizer = optim.SGD(model.parameters(), lr=lr ,weight_decay=weight_decay)
#optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)


print('Testing beginning...')
start_time = time.time()
for epoch in range(1, nbr_epochs+1):
    print('Epoch ', epoch, ':')
    for inputs, target in pre_model.valid_loader:
      inputs, target = inputs.to(device), target.to(device)
      optimizer.zero_grad()
      output = testmodel(inputs)  
      # cache_output.append(target)
      testmodel.testcache_label.append(target)
      loss = loss_fn(output, target)
      loss.backward(retain_graph=True)
      optimizer.step()
    # train(model, pre_model.train_loader, optimizer, epoch)

    # loss, acc = test(model, pre_model.valid_loader)
    
    # save results every epoch
    
    # test_accuracy.append(acc)
    # train_loss.append(loss)
    
end_time = time.time()
print('Testing on ' + str(nbr_epochs) + ' epochs done in ', str(end_time-start_time),' seconds')

Testing beginning...
Epoch  1 :
Epoch  2 :
Epoch  3 :
Epoch  4 :
Epoch  5 :
Epoch  6 :
Epoch  7 :
Epoch  8 :
Epoch  9 :
Epoch  10 :
Testing on 10 epochs done in  26.740750551223755  seconds


In [0]:
len(testmodel.testcache_label)

100

In [0]:
#take out the last set of cache
test_size = 10000
testmodel.testcache_label = testmodel.testcache_label[(len(testmodel.testcache_label)-int(test_size/batchSize)):len(testmodel.testcache_label)]
print(len(testmodel.testcache_label))
testcachelabel = []
for i in testmodel.testcache_label:
 testcachelabel.append(i.detach().numpy())
for i in testcachelabel:
  i.flatten()
testcache_whole_label = []
for i in testcachelabel:
  for m in i:
    testcache_whole_label.append(m)
########################################################cache_whole_label is whole labels of traindata

10


In [0]:
# construct encoder&decoder for each cache (2 in total)
class testEncoderDecoder1(nn.Module):
    def __init__(self, in_dim, out_dim): 
        super(testEncoderDecoder1,self).__init__()
        self.linear1 = torch.nn.Linear(in_dim,50) 
        self.linear2 = torch.nn.Linear(50,4)
        # self.linear3 = torch.nn.Linear(10,4)
        # self.linear4 = torch.nn.Linear(20,10)
        self.testcache_mdim1 = [] #New trained cache
        self.linear3 = torch.nn.Linear(4,out_dim)

    def forward(self,x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        self.testcache_mdim1.append(x)
        x = self.linear3(x)
        x = F.relu(x)
 
        # x = self.linear4(x)
        # x = F.relu(x)


        return x

# for i in CACHE_LIST:
#   i = i.view(-1,150)

testcoder1 = testEncoderDecoder1(150, 150).to(device)


class testEncoderDecoder2(nn.Module):
    def __init__(self, in_dim, out_dim): 
        super(testEncoderDecoder2,self).__init__()
        self.linear1 = torch.nn.Linear(in_dim,50) 
        self.linear2 = torch.nn.Linear(50,4)
        # self.linear3 = torch.nn.Linear(10,4)
        # self.linear4 = torch.nn.Linear(20,10)
        self.testcache_mdim2 = [] #New trained cache
        self.linear3 = torch.nn.Linear(4,out_dim)

    def forward(self,x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        self.testcache_mdim2.append(x) 
        x = self.linear3(x)
        x = F.relu(x)

        # x = self.linear4(x)
        # x = F.relu(x)


        return x

# for i in CACHE_LIST:
#   i = i.view(-1,150)

testcoder2 = testEncoderDecoder2(150, 150).to(device)



In [0]:
#take out the last set of original cache from basic network
testmodel.testcachelin1 = testmodel.testcachelin1[(len(testmodel.testcachelin1)- int(test_size/batchSize)):len(testmodel.testcachelin1)]
testmodel.testcachelin2 = testmodel.testcachelin2[(len(testmodel.testcachelin2)- int(test_size/batchSize)):len(testmodel.testcachelin2)]
print(len(testmodel.testcachelin1))

10


In [0]:
#convert tensors in cache from basic newtork to individual tensor

#convert tensors to numpy
testcachelin1_np = []
for i in testmodel.testcachelin1:
 testcachelin1_np.append(i.detach().numpy())
for i in testcachelin1_np:
  i.flatten()
testcachelin1 = []
for i in testcachelin1_np:
  for m in i:
    testcachelin1.append(m)


testcachelin2_np = []
for i in testmodel.testcachelin2:
 testcachelin2_np.append(i.detach().numpy())
for i in testcachelin2_np:
  i.flatten()
testcachelin2 = []
for i in testcachelin2_np:
  for m in i:
    testcachelin2.append(m)

#convert numpy to tensors
for i in range (len(testcachelin1)):
  testcachelin1[i] = torch.from_numpy(testcachelin1[i])

for i in range (len(testcachelin2)):
  testcachelin2[i] = torch.from_numpy(testcachelin2[i])

In [0]:
# training to reduce dimsension of first cache
epoch_dim = 10
criterion = nn.MSELoss()
optimizer = optim.Adam(testcoder1.parameters(),lr=0.01)
for epoch in range(epoch_dim):
  running_loss = 0.0
  for i in testcachelin1:
    inputs = i.to(device)
    optimizer.zero_grad()
    outputs = testcoder1(inputs)
    loss = criterion(outputs, inputs)
    loss.backward(retain_graph=True)
    running_loss += loss.item()
      


    
print('Finish Training test 1')


Finish Training test 1


In [0]:
# training to reduce dimsension of first cache
epoch_dim = 10
criterion = nn.MSELoss()
optimizer = optim.Adam(testcoder2.parameters(),lr=0.01)
for epoch in range(epoch_dim):
  running_loss = 0.0
  for i in testcachelin2:
    inputs = i.to(device)
    optimizer.zero_grad()
    outputs = testcoder2(inputs)
    loss = criterion(outputs, inputs)
    loss.backward(retain_graph=True)
    running_loss += loss.item()
      


    
print('Finish Training test 2')


Finish Training test 2


In [0]:
#take the last set of reduced-dimension cache respectively
testcache_mdim1 = []
testcache_mdim2 = []
testcache_mdim1 = testcoder1.testcache_mdim1[(len(testcoder1.testcache_mdim1)-int(test_size)):len(testcoder1.testcache_mdim1)]
testcache_mdim2 = testcoder2.testcache_mdim2[(len(testcoder2.testcache_mdim2)-int(test_size)):len(testcoder2.testcache_mdim2)]


In [0]:
testcache_mdim1[0].size()

torch.Size([4])

In [0]:
#convert reduced-dimension testcache to numpy 
testcache_mdim1_np = []
testcache_mdim2_np = []
for i in range(len(testcache_mdim1)):
  testcache_mdim1_np.append(testcache_mdim1[i].detach().numpy())

for i in range(len(testcache_mdim2)):
  testcache_mdim2_np.append(testcache_mdim2[i].detach().numpy())
######################################

In [0]:
type(testcache_mdim2_np[0])

numpy.ndarray

In [0]:
#normalization

def normalize(data):
    m = np.mean(data)
    mx = max(data)
    mn = min(data)
    return [(float(i) - m) / (mx - mn) for i in data]


for i in range(len(cache_mdim1_np)):
  cache_mdim1_np[i] = normalize(cache_mdim1_np[i])

for i in range(len(cache_mdim2_np)):
  cache_mdim2_np[i] = normalize(cache_mdim2_np[i])

for i in range(len(testcache_mdim1_np)):
   testcache_mdim2_np[i] = normalize(testcache_mdim2_np[i])

for i in range(len(testcache_mdim2_np)):
   testcache_mdim1_np[i] = normalize(testcache_mdim1_np[i])




In [0]:
(cache_mdim2_np[0])

[-0.25, 0.75, -0.25, -0.25]

In [0]:
cache_mdim2_np

In [0]:
from sklearn.neighbors import NearestNeighbors
neigh_layer1 = NearestNeighbors(n_neighbors=5)
neigh_layer1.fit(cache_mdim1_np) #训练第一层的cache，是个预测模型
neigh_layer1_list = []
for i in range(len(testcache_mdim1_np)):
  neigh_layer1_list.append(neigh_layer1.kneighbors([testcache_mdim1_np[i]]))



# neigh_layer1_list的长度应该是10000，每一个里面有2个元素，前者是对应于五个最近邻居的距离，后五个对应邻居的索引

In [0]:
neigh_layer1_list[0]

In [0]:
times0 = 0
times1 = 0
list_confidence0 = []
list_confidence1 = []
distance0 = 0
distance1 = 0
#initialize
for i in range(len(neigh_layer1_list)):
  for j in range(5):
    if cache_whole_label[neigh_layer1_list[i][1][0][j]] == 1:
       times1 = times1 + 1
       distance1 = distance1 + 1/neigh_layer1_list[i][0][0][j]
    if cache_whole_label[neigh_layer1_list[i][1][0][j]] == 0:
       times0 = times0 + 1
       distance0 = distance0 + 1/neigh_layer1_list[i][0][0][j]
  confidence0 = times0/5*distance0
  confidence1 = times1/5*distance1
  list_confidence0.append(confidence0)
  list_confidence1.append(confidence1)
  #delete to 0
  confidence0 = 0
  confidence1 = 0
  distance0 = 0
  distance1 = 0
  times0 = 0
  times1 = 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app


In [0]:
con_label = []
for i in range(len(list_confidence0)):
  if list_confidence0[i] > list_confidence1[i]:
    con_label.append([list_confidence0[i],"0"])
  if list_confidence0[i] < list_confidence1[i]:
    con_label.append([list_confidence1[i],"1"])


  

In [0]:
len(con_label)

10000

In [0]:
con_label[0]

[4.974836812686197, '1']